In [ ]:
from io import BytesIO
import cartopy.crs as ccrs 
import numpy as np
import requests
import xarray as xr
import hvplot.xarray

def download_data(url, **kwargs):
    r = requests.get(url)
    buf = BytesIO(r.content)
    return xr.open_dataset(buf, **kwargs)    

# Generate data remotely
cmda_url = 'http://ec2-52-53-95-229.us-west-1.compute.amazonaws.com:8080/svc/correlationMap'
model1, var1, pres1 = 'GFDL_ESM2G', 'cli', 500
model2, var2, pres2 = 'GFDL_ESM2G', 'rlutcs', -999999
laggedTime = 0
width, height = 1000, 500
v1 = f'{model1}:{var1}'
v2 = f'{model2}:{var2}'
query = dict(model1=model1, var1=var1, pres1=pres1, model2=model2, var2=var2, pres2=pres2, lonS=0, lonE=360, 
             latS=-90, latE=90, timeS=200401, timeE=200412, laggedTime=laggedTime, purpose='')
r = requests.get(cmda_url, params=query)
data_url = r.json()['dataUrl']

# Download data into xarray Dataset object
ds = download_data(data_url).rename(index='EOF')
ds

In [ ]:
def plot_data(data, **kwargs):
    ax = plt.axes(projection=ccrs.PlateCarree())
    ax.set_global(); ax.coastlines();
    return data.plot(ax=ax, transform=ccrs.PlateCarree(), **kwargs)
ds.patterns.hvplot.quadmesh('lon', 'lat', coastline=True)

In [ ]:
ds.tser.hvplot.line(x='time', y='tser')